<a href="https://colab.research.google.com/github/levidasilvalima/RNA-PP1/blob/master/RedesNeuraisArtificiais_Projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# IMPORTAÇÕES

from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# CRIAÇÃO DO DRIVE

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [92]:
# iMPORTAR CSV

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Manaus.csv', sep=';', encoding='ISO-8859-1')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,15,22,24,30,31,32,33,35) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [93]:
# TÓPICO - 2..1 Visão Geral dos Casos Confirmados

filter = df['_classificacao'] == "Confirmado"
df = df[filter] # Filtrar por casos confirmados
print(df.columns.size, "Valores descrevem cada exemplo")
print("Sendo eles: \n", df.columns.values)

print("\nEm manaus há", len(df), "casos confirmados")

ordered = df.sort_values(by='_dt_notificacao', ascending=True)
notNullFilter = pd.notnull(ordered["_dt_notificacao"]) 
ordered = ordered[notNullFilter]
initialDate = ordered.head(1)['_dt_notificacao'].values[0]
endDate = ordered.iloc[::-1].head(1)['_dt_notificacao'].values[0]
print("\nO periodo de tempo da base de dados vai de", initialDate, "a", endDate)


36 Valores descrevem cada exemplo
Sendo eles: 
 ['_idade' '_faixa etária' '_sexo' '_bairro' '_classificacao'
 '_comorb_renal' '_comorb_diabetes' '_comorb_imuno' '_comorb_cardio'
 '_conclusao' '_dt_notificacao' '_taxa' '_dt_evolucao' '_raca'
 '_dt_sintomas' '_criterio' '_tipo_teste' '_sintoma_garganta'
 '_sintoma_dispneia' '_sintoma_febre' '_sintoma_tosse' '_sintoma_outros'
 '_etnia' '_profiss_saude' '_srag' '_se_notificacao' '_distrito'
 '_bairro_mapa' '_comorb_respiratoria' '_comorb_cromossomica'
 '_comorb_hepatica' '_comorb_neurologica' '_comorb_hemato'
 '_comorb_obessidade' '_origem' '_evolução']

Em manaus há 36671 casos confirmados

O periodo de tempo da base de dados vai de 01/04/2020 a 31/07/2020


In [94]:
# Limpando dataset

df = df.drop(columns=['_comorb_renal', 
                      '_comorb_diabetes', 
                      '_comorb_imuno', 
                      '_comorb_cardio', 
                      '_comorb_respiratoria', 
                      '_comorb_cromossomica', 
                      '_comorb_hepatica', 
                      '_comorb_neurologica', 
                      '_comorb_hemato', 
                      '_comorb_obessidade']) # Apagar atributos relativos a comorbidades
df = df.drop(columns=['_sintoma_garganta', 
                      '_sintoma_dispneia', 
                      '_sintoma_febre', 
                      '_sintoma_tosse', 
                      '_sintoma_outros']) # Apagar atributos relativos a sintomas
df = df.drop(columns=['_etnia', '_raca']) # Apagar atributos relativos a etnia
df = df.drop(columns=['_profiss_saude']) # Apagar atributos relativos a profissao
df = df.drop(columns=['_dt_evolucao',
                      '_dt_sintomas']) # Apagar atributos data exceto a de notificação
df = df.drop(columns=['_origem']) # Apagar atributos relativos a origem

df = df.dropna(subset=df.columns.values) # Apagar linhas com atributos nulos
df
# df.isnull().sum()
# df = df.dropna(inplace=True)


,_idade,_faixa etária,_sexo,_bairro,_classificacao,_conclusao,_dt_notificacao,_taxa,_criterio,_tipo_teste,_srag,_se_notificacao,_distrito,_bairro_mapa,_evolução
105779,54.0,50a - 59a,M,CIDADE DE DEUS,Confirmado,Recuperado,18/06/2020,"0,117699678",Laboratorial,RT-PCR,COVID-19,25.0,NORTE,CIDADE DE DEUS,Internado
105780,30.0,30a - 39a,M,NOVA CIDADE,Confirmado,Recuperado,29/04/2020,"0,13857325",Laboratorial,TESTE RÁPIDO - ANTÍGENO,COVID-19,18.0,NORTE,NOVA CIDADE,Internado
105781,54.0,50a - 59a,M,NOVO ISRAEL,Confirmado,Recuperado,18/04/2020,"0,490725292",Laboratorial,TESTE RÁPIDO - ANTICORPO,COVID-19,16.0,NORTE,NOVO ISRAEL,Internado
105782,52.0,50a - 59a,M,CIDADE NOVA,Confirmado,Recuperado,20/04/2020,"0,068152389",Clínico,TESTE RÁPIDO - ANTICORPO,SRAG não especificado,17.0,NORTE,CIDADE NOVA,Internado
105786,55.0,50a - 59a,F,MORRO DA LIBERDADE,Confirmado,Recuperado,18/06/2020,"0,749793807",Laboratorial,TESTE RÁPIDO - ANTICORPO,COVID-19,25.0,SUL,MORRO DA LIBERDADE,Internado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107193,0.0,< 1a,M,NOVO ALEIXO,Confirmado,Recuperado,28/06/2020,"0,085452557",Laboratorial,TESTE RÁPIDO - ANTICORPO,COVID-19,27.0,NORTE,NOVO ALEIXO,Em tratamento domiciliar
107196,14.0,10a - 14a,F,DA PAZ,Confirmado,Recuperado,25/06/2020,"0,543389665",Laboratorial,TESTE RÁPIDO - ANTÍGENO,COVID-19,26.0,OESTE,DA PAZ,Em tratamento domiciliar
107202,5.0,5a - 9a,F,REDENCAO,Confirmado,Recuperado,18/06/2020,"0,234763828",Laboratorial,TESTE RÁPIDO - ANTÍGENO,COVID-19,25.0,OESTE,REDENÇÃO,Internado em UTI
107204,70.0,70a - 79a,F,DOM PEDRO I,Confirmado,Recuperado,02/06/2020,"0,483629153",Laboratorial,TESTE RÁPIDO - ANTÍGENO,COVID-19,23.0,OESTE,DOM PEDRO,Em tratamento domiciliar
